# REF Assessment Notebook

### 31st May 2021

### Neil D. Lawrence

This notebook is a helper for reviewing ref papers.


In [ ]:
import ref

In [ ]:
df = ref.assess.data()

In [ ]:
ref.assess.query(df, 230808)

In [ ]:
ref.assess.view(df)